## Proyecto 2. Introducción al análisis de datos
En este caso Synergy Logistics nos provee una base de datos sobre las rutas empleadas durante la importación y exportación de productos desde el año 2015. Además, se nos solicita que realicemos un análisis para guiar la estrategia operativa 2021. Para ello, hay que enfocarnos en las siguientes tareas.

#### Opción 1) Rutas de importación y exportación. 
Synergy logistics está considerando la posibilidad de enfocar sus esfuerzos en las 10 rutas más demandadas. Acorde a los flujos de importación y exportación, ¿cuáles son esas 10 rutas? ¿le conviene implementar esa estrategia? ¿por qué?

#### Lo primero es importar la base de datos

In [2]:
import numpy as np
"""Empleamos el módulo pandas para convertir el archivo csv en un DataFrame"""
import pandas as pd
datos=pd.read_csv(r"C:\Users\flore\Desktop\synergy_logistics_database.csv")
"""Ahora vamos a analizar el DataFrame generado para darnos una idea de los datos"""
datos.head()
#"""Además queremos saber las dimensiones del mismo"""
#datos.shape

,register_id,direction,origin,destination,year,date,product,transport_mode,company_name,total_value
0,1,Exports,Japan,China,2015,31/01/15,Cars,Sea,Honda,33000000
1,2,Exports,Japan,China,2015,01/02/15,Cars,Sea,Honda,16000000
2,3,Exports,Japan,China,2015,02/02/15,Cars,Sea,Honda,29000000
3,4,Exports,Japan,China,2015,03/02/15,Cars,Sea,Honda,14000000
4,5,Exports,Japan,China,2015,04/02/15,Cars,Sea,Honda,17000000


Ahora, vamos a definir una ruta en función del país de origen, del país de destino y del modo de transporte. Ahora, para eso hay que conocer los múltiples países de origen, de destino y los modos de transporte.

In [3]:
#Los valores únicos dentro de la columna origin:
paises_origen=datos['origin'].unique()
#print(paises_origen)
#Los valores únicos dentro de la columna destination
paises_destino=datos['destination'].unique()
#print(paises_destino)
#Los posibles valores de modo de transporte:
modos_de_transporte=datos['transport_mode'].unique()
#print(modos_de_transporte)
#Las posibles direcciones:
direcciones=datos['direction'].unique()
print(direcciones)

['Exports' 'Imports']


In [4]:
#Hacemos la distinción entre las exportaciones y las importaciones.
exportaciones=datos[datos['direction']=='Exports']
importaciones=datos[datos['direction']=='Imports']

Además, también hay que hacer la distinción entre la dirección: se trata de exportaciones y de importaciones. Ahora, queremos saber cuál de todas esas rutas son las más demandadas, es decir, cuál de esas rutas tiene más registros asociados.

In [8]:
#Definimos una función que reciba el nombre de dos paises, el medio de transporte, y la dirección; y nos devuelva el número de registros asociados. 
def demanda(datos,pais_origen, pais_destino, transporte):
    nombre_ruta=pais_origen+"-"+pais_destino+'-'+transporte
    #Filtramos por los datos que queremos.
    data=datos[(datos['origin']==pais_origen) & (datos['destination']==pais_destino) & (datos['transport_mode']==transporte) ]
    #Buscamos que nos de la cantidad de registros y el valor total de la ruta
    num_registros=data.shape[0]
    valor= data['total_value'].sum()
    return nombre_ruta, num_registros, valor

In [9]:
demanda(exportaciones,'Japan','China','Sea')

('Japan-China-Sea', 196, 2550000000)

In [11]:
#Ahora, vamos a aplicar esta función sobre todas las combinaciones posibles de rutas, pero únicamente para exportaciones.
Rutas_exp=[]
for origen in paises_origen:
    for destino in paises_destino:
        for modo in modos_de_transporte:
            valores=demanda(exportaciones,origen,destino,modo)
            if valores[1]==0:
                continue
            else:
                Rutas_exp.append(valores)

Ya contamos con una lista que posee toda la información de estas rutas asociadas a las exportaciones, si ordenamos por el número de transacciones asociadas, podemos ver cuáles son las rutas de exportación más demandadas.

In [39]:
Rutas_exp_ordenadas_por_demanda=sorted(Rutas_exp,reverse=True, key=lambda x:x[1])
Rutas_exp_10_demanda=Rutas_exp_ordenadas_por_demanda[0:10]
ganancias_exp_10_demanda=sum([x[2] for x in Rutas_exp_10_demanda])
print([x[0] for x in Rutas_exp_10_demanda])

['South Korea-Vietnam-Sea', 'USA-Netherlands-Sea', 'Netherlands-Belgium-Road', 'China-Mexico-Air', 'Japan-Brazil-Sea', 'Germany-France-Road', 'South Korea-Japan-Sea', 'Australia-Singapore-Sea', 'Canada-Mexico-Rail', 'China-Spain-Air']


In [40]:
#Pero veamos si de verdad esas son las rutas que más proveen ingresos a la empresa al ordenar por el valor asociado a las rutas.
Rutas_exp_ordenadas_por_ganancia=sorted(Rutas_exp,reverse=True, key=lambda x:x[2])
Rutas_exp_10_ganancia=Rutas_exp_ordenadas_por_ganancia[0:10]
ganancias_exp_10_ganancia=sum([x[2] for x in Rutas_exp_10_ganancia])
print([x[0] for x in Rutas_exp_10_ganancia])

['China-Mexico-Air', 'Canada-Mexico-Rail', 'South Korea-Vietnam-Sea', 'France-United Kingdom-Sea', 'South Korea-Japan-Sea', 'China-South Korea-Rail', 'USA-Mexico-Rail', 'France-Belgium-Road', 'China-Germany-Air', 'China-USA-Air']


Si nos concentramos en las 10 rutas con más demandadas, estaríamos generando:

In [41]:
print(ganancias_exp_10_demanda)

45058657000


Si nos concentramos en las 10 rutas con mayores ingresos, generaríamos:

In [42]:
print(ganancias_exp_10_ganancia)

58647007000


Por lo tanto, no sería buena idea concentrarse en las 10 rutas de exportación más demandadas, sería mejor concentrarse en las 10 rutas de exportación con mayores ganancias, las cuales nos dejarían 13,588,350,000 pesos más que concentrarnos en las 10 rutas más demandadas.
Sin embargo, se nos pide darles a conocer cuáles son estas 10 rutas:
##### South Korea-Vietnam-Exports-Sea
##### USA-Netherlands-Exports-Sea
##### Netherlands-Belgium-Exports-Road
##### China-Mexico-Exports-Air
##### Japan-Brazil-Exports-Sea
##### Germany-France-Exports-Road
##### South Korea-Japan-Exports-Sea
##### Australia-Singapore-Exports-Sea
##### Singapore-Thailand-Imports-Sea
##### Canada-Mexico-Exports-Rail

Ahora, en el caso de las importaciones:

In [49]:
#Ahora, vamos a aplicar esta función sobre todas las combinaciones posibles de rutas, pero únicamente para exportaciones.
Rutas_imp=[]
for origen in paises_origen:
    for destino in paises_destino:
        for modo in modos_de_transporte:
            valores=demanda(importaciones,origen,destino,modo)
            if valores[1]==0:
                continue
            else:
                Rutas_imp.append(valores)

In [50]:
Rutas_imp_ordenadas_por_demanda=sorted(Rutas_imp,reverse=True, key=lambda x:x[1])
Rutas_imp_10_demanda=Rutas_imp_ordenadas_por_demanda[0:10]
ganancias_imp_10_demanda=sum([x[2] for x in Rutas_imp_10_demanda])
print([x[0] for x in Rutas_imp_10_demanda])

['Singapore-Thailand-Sea', 'Germany-China-Sea', 'China-Japan-Air', 'Japan-Mexico-Sea', 'Malaysia-Thailand-Rail', 'China-Thailand-Road', 'Spain-Germany-Road', 'China-United Arab Emirates-Sea', 'Brazil-China-Sea', 'USA-Thailand-Sea']


In [51]:
Rutas_imp_ordenadas_por_ganancia=sorted(Rutas_imp,reverse=True, key=lambda x:x[2])
Rutas_imp_10_ganancia=Rutas_imp_ordenadas_por_ganancia[0:10]
ganancias_imp_10_ganancia=sum([x[2] for x in Rutas_imp_10_ganancia])
print([x[0] for x in Rutas_imp_10_ganancia])

['Singapore-Thailand-Sea', 'Japan-Mexico-Sea', 'Malaysia-Thailand-Rail', 'China-United Arab Emirates-Sea', 'China-Japan-Air', 'China-Thailand-Road', 'Japan-United Arab Emirates-Sea', 'Mexico-USA-Rail', 'Spain-Germany-Road', 'Germany-Mexico-Sea']


In [53]:
print(ganancias_imp_10_demanda)

26150000000


In [54]:
print(ganancias_imp_10_ganancia)

28943000000


En este caso, de concentrarnos en las 10 rutas de importación con mayor ganancia, obtendríamos 2,793,000,000 más que de concentrarnos en las 10 rutas de importación con mayor demanda. Las 10 rutas de importación más demandadas son:

##### Singapore-Thailand-Sea
##### Germany-China-Sea
##### China-Japan-Air
##### Japan-Mexico-Sea
##### Malaysia-Thailand-Rail
##### China-Thailand-Road
##### Spain-Germany-Road
##### China-United Arab Emirates-Sea
##### Brazil-China-Sea
##### USA-Thailand-Sea

### Opción 2) Medio de transporte utilizado.
¿Cuáles son los 3 medios de transporte más importantes para Synergy logistics considerando el valor de las importaciones y exportaciones? ¿Cuál es medio de transporte que podrían reducir? 

En este caso, vamos a definir una función que para un tipo de transporte dado, nos devuelva el valor total de las importaciones o exportaciones asociados. El transporte a reducir será aquel que nos deje menos ingresos. 

In [85]:
def Ganancia_transporte(datos,transporte):
    #Filtramos por el tipo de transporte:
    data_transport=datos[datos['transport_mode']==transporte]
    ganancia=data_transport['total_value'].sum()
    return transporte, ganancia

In [86]:
Ganancias_exp_transporte=[]
for modo in modos_de_transporte:
    Ganancias_exp_transporte.append(Ganancia_transporte(exportaciones,modo))
#Ordenamos
Ganancias_exp_transporte_ordenada=sorted(Ganancias_exp_transporte,reverse=True, key=lambda x:x[1])
print(Ganancias_exp_transporte_ordenada)

[('Sea', 65592622000), ('Rail', 34505043000), ('Air', 32785147000), ('Road', 27280486000)]


In [87]:
Ganancias_imp_transporte=[]
for modo in modos_de_transporte:
    Ganancias_imp_transporte.append(Ganancia_transporte(importaciones,modo))
#Ordenamos
Ganancias_imp_transporte_ordenada=sorted(Ganancias_imp_transporte,reverse=True, key=lambda x:x[1])
print(Ganancias_imp_transporte_ordenada)

[('Sea', 34938000000), ('Rail', 9123000000), ('Road', 5990000000), ('Air', 5477000000)]


##### Los 3 medios de transporte  más importantes son Sea, Rail y Road, para las importaciones. El medio de transporte ha reducir podría ser "Air".

### Opción 3) Valor total de importaciones y exportaciones. 

Si Synergy Logistics quisiera enfocarse en los países que le generan el 80% del valor de las exportaciones e importaciones ¿en qué grupo de países debería enfocar sus esfuerzos?

El 80% de las exportaciones e importaciones se pueden obtener a partir de diferentes rutas, como se describió en el punto 1, sin embargo, no se nos pide que nos concentremos en vías de transporte, sólamente en los paises. Por lo que vamos a abarcar únicamente a los países de origen y de destino, para lo que hay que generar una función parecida a la de la opción 1.

In [61]:
#Definimos una función que reciba el nombre del pais y nos devuelva el número de registros asociados dependiendo de la dirección
def paises_porcentajes(datos,pais,direccion):
    #Filtramos por los datos que queremos
    data=datos[(datos[direccion]==pais)]
    #Buscamos que nos de la cantidad de registros y el valor total de la ruta
    porcentaje= data['total_value'].sum() / datos['total_value'].sum()
    return pais, porcentaje

In [67]:
paises_porcentajes(exportaciones,'Germany','origin')

('Germany', 0.07082292348900059)

In [73]:
#Ahora, vamos a iterar sobre los paises de origen para exportaciones, vamos a calcular los porcentajes y luego vamos a ordenarlos y sumarlos 
#hasta obtener el porcentaje del 80% del valor total de las exportaciones, es decir, 0.80.

Porcentajes_exp=[]
for origen in paises_origen:
    if paises_porcentajes(exportaciones,origen,'origin')[1]==0:
        continue
    else: 
        Porcentajes_exp.append(paises_porcentajes(exportaciones,origen,'origin'))
Porcentajes_exp_ordenados=sorted(Porcentajes_exp, reverse=True, key=lambda x:x[1])
#print(Porcentajes_ordenados)
porcentaje_actual=0
Paises_exp_origen_porcentaje=[]
Avance=[]
for porcentaje in Porcentajes_exp_ordenados:
    if porcentaje_actual < 0.80:
        Paises_exp_origen_porcentaje.append(porcentaje[0])
        porcentaje_actual+= porcentaje[1]
        Avance.append(porcentaje_actual)
    else:
        continue

In [76]:
print(Paises_exp_origen_porcentaje)
print(Avance)

0.822704331425543
['China', 'France', 'USA', 'South Korea', 'Russia', 'Japan', 'Germany', 'Canada']
[0.20589639706345209, 0.32211735550050924, 0.43672105203527967, 0.5280100438491221, 0.6105695326029064, 0.6851744898509771, 0.7559974133399776, 0.822704331425543]


Así en cuanto a exportaciones,los paises de origen involucrados en las rutas que generan el 80% de los ingresos son:

##### China
##### France
##### USA
##### South Korea
##### Russia
##### Japan
##### Germany
##### Canada

Ahora, vamos a ver los paises de destino involucrados en las exportaciones que generan el 80% de los ingresos:

In [77]:
Porcentajes_exp=[]
for destino in paises_destino:
    if paises_porcentajes(exportaciones,destino,'destination')[1]==0:
        continue
    else: 
        Porcentajes_exp.append(paises_porcentajes(exportaciones,destino,'destination'))
Porcentajes_exp_ordenados=sorted(Porcentajes_exp, reverse=True, key=lambda x:x[1])
#print(Porcentajes_ordenados)
porcentaje_actual=0
Paises_exp_destino_porcentaje=[]
Avance=[]
for porcentaje in Porcentajes_exp_ordenados:
    if porcentaje_actual < 0.80:
        Paises_exp_destino_porcentaje.append(porcentaje[0])
        porcentaje_actual+= porcentaje[1]
        Avance.append(porcentaje_actual)
    else:
        continue

In [79]:
print(Paises_exp_destino_porcentaje)
print(Avance)

0.8345596879504816
['Mexico', 'USA', 'Belgium', 'Germany', 'China', 'Japan', 'United Kingdom', 'Brazil', 'South Korea', 'Vietnam', 'Italy', 'Spain', 'France']
[0.17678841128758474, 0.25895086151385316, 0.34073851301438607, 0.41131305250719796, 0.48161625018485815, 0.5364175193245583, 0.5873072868417082, 0.6341845433277729, 0.6798983122837543, 0.7228357835138984, 0.7609801591373324, 0.7980310382969261, 0.8345596879504816]


Los paises de destino que aportan el 80% de los ingresos en exportaciones son:

##### China
##### France
##### USA
##### South Korea
##### Russia
##### Japan
##### Germany
##### Canada

###### Ahora, realizamos lo mismo para las importaciones

In [80]:
Porcentajes_imp=[]
for origen in paises_origen:
    if paises_porcentajes(importaciones,origen,'origin')[1]==0:
        continue
    else: 
        Porcentajes_imp.append(paises_porcentajes(importaciones,origen,'origin'))
Porcentajes_imp_ordenados=sorted(Porcentajes_imp, reverse=True, key=lambda x:x[1])
#print(Porcentajes_ordenados)
porcentaje_actual=0
Paises_imp_origen_porcentaje=[]
Avance=[]
for porcentaje in Porcentajes_imp_ordenados:
    if porcentaje_actual < 0.80:
        Paises_imp_origen_porcentaje.append(porcentaje[0])
        porcentaje_actual+= porcentaje[1]
        Avance.append(porcentaje_actual)
    else:
        continue

In [82]:
print(Paises_imp_origen_porcentaje)
print(Avance)

0.8276004898429621
['China', 'Japan', 'USA', 'Mexico', 'Germany', 'Singapore', 'South Korea', 'Malaysia']
[0.22030327042212938, 0.36606756951447916, 0.46135283100417807, 0.5445721077654516, 0.6211100705950151, 0.6934519521682755, 0.7634886903904337, 0.8276004898429621]


Los países de origen con mayor contribución al 80% de las importaciones son:

##### China
##### Japan
##### USA
##### Mexico
##### Germany
##### Singapore
##### South Korea
##### Malaysia

Ahora, con los paises de destino de las importaciones:

In [83]:
Porcentajes_imp=[]
for destino in paises_destino:
    if paises_porcentajes(importaciones,destino,'destination')[1]==0:
        continue
    else: 
        Porcentajes_imp.append(paises_porcentajes(importaciones,destino,'destination'))
Porcentajes_imp_ordenados=sorted(Porcentajes_imp, reverse=True, key=lambda x:x[1])
#print(Porcentajes_ordenados)
porcentaje_actual=0
Paises_imp_destino_porcentaje=[]
Avance=[]
for porcentaje in Porcentajes_imp_ordenados:
    if porcentaje_actual < 0.80:
        Paises_imp_destino_porcentaje.append(porcentaje[0])
        porcentaje_actual+= porcentaje[1]
        Avance.append(porcentaje_actual)
    else:
        continue

In [88]:
print(Paises_imp_destino_porcentaje)
print(Avance)

['Thailand', 'Mexico', 'United Arab Emirates', 'Japan', 'Germany', 'USA']
[0.2475327762570235, 0.4077042212937617, 0.5490383230082121, 0.6539943812130817, 0.7460020170004322, 0.8136795850741968]


Los paises de destino que aportan el 80% de las importaciones son:

##### Thailand
##### Mexico
##### United Arab Emirates
##### Japan
##### Germany
##### USA